In [1]:
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
import pinecone



c:\Users\pc\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = "c285b984-c6ac-4da9-9dc7-2f1d3416e579"

In [2]:
# extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()
    return documents

In [4]:
extracted_data = load_pdf("data/")

FileNotFoundError: Directory not found: 'data/'

In [6]:
# to check the output
extracted_data

NameError: name 'extracted_data' is not defined

In [7]:
# create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap = 20)
    text_chuncks = text_splitter.split_documents(extracted_data)

    return text_chuncks

In [8]:
text_chunks = text_split(extracted_data)
print("length of chunks",len(text_chunks))

length of chunks 7020


In [15]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [43]:
embeddings = download_hugging_face_embeddings()

In [17]:
# check details of embedding model
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [108]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [19]:
query_result

[-0.034477267414331436,
 0.031023206189274788,
 0.006734929047524929,
 0.026108982041478157,
 -0.03936203196644783,
 -0.16030243039131165,
 0.06692398339509964,
 -0.006441489793360233,
 -0.04745049402117729,
 0.014758839271962643,
 0.07087527960538864,
 0.05552763119339943,
 0.01919335499405861,
 -0.026251323521137238,
 -0.010109569877386093,
 -0.02694045566022396,
 0.022307392209768295,
 -0.02222662791609764,
 -0.14969263970851898,
 -0.017493031919002533,
 0.007676294539123774,
 0.054352253675460815,
 0.0032544503919780254,
 0.031725917011499405,
 -0.08462144434452057,
 -0.029405998066067696,
 0.051595620810985565,
 0.048124048858881,
 -0.00331486901268363,
 -0.058279212564229965,
 0.041969265788793564,
 0.022210726514458656,
 0.12818878889083862,
 -0.02233896590769291,
 -0.011656217277050018,
 0.06292834132909775,
 -0.032876402139663696,
 -0.09122605621814728,
 -0.03117532841861248,
 0.05269954353570938,
 0.047034818679094315,
 -0.08420311659574509,
 -0.030056139454245567,
 -0.020744

In [109]:
import os
from langchain_pinecone import PineconeVectorStore
os.environ['PINECONE_API_KEY'] = "c285b984-c6ac-4da9-9dc7-2f1d3416e579"
index_name = "medical-chatbot"
PINECONE_API_KEY = "c285b984-c6ac-4da9-9dc7-2f1d3416e579"

In [49]:
docsearch = PineconeVectorStore.from_texts(
        [t.page_content for t in text_chunks],
        index_name=index_name,
        embedding=embeddings
    )

In [110]:
#If we already have an index we can load it like this
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [111]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [112]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [9]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-664b28cb-2a5e2e4802fd0e8436993fb3;7d00e26e-44a1-46eb-a2fa-ecaf6034c678)

Repository Not Found for url: https://huggingface.co/api/models/model/llama-2-7b-chat.ggmlv3.q4_0.bin/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid username or password.

In [8]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

NameError: name 'llm' is not defined

In [7]:
print('test')

test
